<a href="https://colab.research.google.com/github/akshaypt7/amazon_sales_prediction/blob/main/Using_entity_embeddings_in_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
[open,store_index,day_of_week,promo,year,month,day,store_state] - first page

{1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5}

In [ ]:
[store_embedding, dow_embedding, year_embedding,
                     month_embedding, day_embedding, german_states_embedding]

In [ ]:
[asin_embedding, hours_embedding, dayofweek_embedding,
                  month_embedding, day_embedding, week_embedding]

In [ ]:
['Asin', 'Item Promo Discount', 'price', 'hour', 'Invoice Amount',
       'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed']

In [ ]:
{0: 0, 3: 1, 9: 2, 6: 3, 8: 4, 7: 5}

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 7.4 MB/s 
     |████████████████████████████████| 1.2 MB 16.2 MB/s 
     |████████████████████████████████| 188 kB 44.0 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 51 kB 357 kB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [ ]:
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers import Dense, Flatten, Dropout, Input, Embedding, Reshape
from keras.layers import concatenate
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

from sklearn.model_selection import train_test_split
import math
import pandas as pd

In [ ]:
infile = open('/content/drive/MyDrive/bluebook/df_main.pkl','rb')
df_main = pickle.load(infile)

In [ ]:
df_main = add_datepart(df_main,'date')

In [ ]:
cat = ['Asin',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'hour','Month','Week', 'Day','Dayofweek','Dayofyear']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
enc = OrdinalEncoder()
enc.fit(df_main[cat])
df_main[cat]= enc.transform(df_main[cat])

In [ ]:
condition = (df_main.Year < 2020) | (df_main.Month < 8)
train_idx = np.where(condition)[0]
valid_idx = np.where(~condition)[0]

In [ ]:
X =df_main.drop(['Quantity'],axis=1)

In [ ]:
X_train = df_main.drop(['Quantity'],axis=1).iloc[train_idx]
y_train = df_main['Quantity'].iloc[train_idx]

X_valid = df_main.drop(['Quantity'],axis=1).iloc[valid_idx]
y_valid = df_main['Quantity'].iloc[valid_idx]

In [ ]:
saved_embeddings_fname = '/content/gdrive/MyDrive/bluebook/embeddings.pickle'

In [ ]:
def embed_features(X, saved_embeddings_fname):
    f_embeddings = open(saved_embeddings_fname, "rb")
    embeddings = pickle.load(f_embeddings)

    index_embedding_mapping = {0: 0, 3: 1, 9: 2, 6: 3, 8: 4, 7: 5}
    X_embedded = []

    (num_records, num_features) = X.shape
    for record in X:
        embedded_features = []
        for i, feat in enumerate(record):
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return np.array(X_embedded)

In [ ]:
X_emb = embed_features(X.values,saved_embeddings_fname)

In [ ]:
X_emb

array([[-2.73080558e-01,  2.78832138e-01, -2.88404405e-01, ...,  0.00000000e+00,  0.00000000e+00,  1.60971840e+09],
       [-3.27337712e-01,  3.36779356e-01,  3.31822723e-01, ...,  0.00000000e+00,  0.00000000e+00,  1.61006400e+09],
       [-2.22492591e-01, -2.76432276e-01, -3.15241039e-01, ...,  0.00000000e+00,  0.00000000e+00,  1.61015040e+09],
       ...,
       [-3.00294220e-01,  2.89772391e-01,  2.93079853e-01, ...,  0.00000000e+00,  0.00000000e+00,  1.57239360e+09],
       [-3.28152686e-01,  2.92389959e-01, -2.12380916e-01, ...,  0.00000000e+00,  0.00000000e+00,  1.57248000e+09],
       [ 3.21957134e-02, -3.75702754e-02,  4.75607626e-02, ...,  0.00000000e+00,  0.00000000e+00,  1.57248000e+09]])

This works - res_list = list(itemgetter(*index_list)(test_list)) 

In [ ]:
X_train = list(itemgetter(*train_idx)(X_emb)) 
X_valid = list(itemgetter(*valid_idx)(X_emb)) 

In [ ]:

y_train = df_main['Quantity'].iloc[train_idx]

y_valid = df_main['Quantity'].iloc[valid_idx]

The current status of ours is since the above X_emb is a list, instead of iloc, we need to find someother way to select the values from the index list we have.

In [ ]:
l = [3,2,1,3,4]

In [ ]:
l[0,3]

TypeError: ignored

refer - https://stackoverflow.com/questions/25431850/passing-a-list-of-indices-to-another-list-in-python-correct-syntax